In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import time as T
import pyproj

In [2]:
df = pd.read_csv('data.csv')
df

,ID,user_id,latitude,longitude,flag,time
0,3865408,0,30.364965,-81.675575,True,2022-09-21 02:10:55
1,3865411,0,30.364965,-81.675575,True,2022-09-21 02:11:44
2,7902772,0,30.459910,-81.662560,True,2022-10-10 18:41:33
3,7902773,0,30.459920,-81.662550,True,2022-10-10 18:41:48
4,7902774,0,30.551570,-81.820950,True,2022-10-10 18:54:04
...,...,...,...,...,...,...
6715364,13980039,34497,29.381240,-82.233210,True,2022-10-29 20:39:33
6715365,13980040,34497,29.381230,-82.233220,True,2022-10-29 20:39:40
6715366,13980047,34497,29.381230,-82.233250,True,2022-10-30 20:39:30
6715367,13980048,34497,29.381230,-82.233220,True,2022-10-30 20:39:39


In [3]:
sp = pd.read_csv('staypoints.csv')
sp

,id,user_id,started_at,finished_at,geom
0,0,0,2022-10-10 18:41:33-04:00,2022-10-10 18:54:04-04:00,POINT (-81.6625549999999976 30.4599150000000023)
1,1,2,2022-09-06 09:15:08-04:00,2022-09-06 09:53:50-04:00,POINT (-81.6775222899999989 29.6550346550000015)
2,2,2,2022-09-06 21:34:07-04:00,2022-09-06 22:09:29-04:00,POINT (-81.6840588550000177 29.6517638699999999)
3,3,2,2022-09-06 22:09:29-04:00,2022-09-06 22:19:08-04:00,POINT (-81.6582657950000055 29.6534941799999991)
4,4,2,2022-09-06 22:22:07-04:00,2022-09-06 22:31:37-04:00,POINT (-81.6582655899999992 29.6534943599999998)
...,...,...,...,...,...
432635,432635,34483,2022-10-27 06:16:00-04:00,2022-10-27 06:44:18-04:00,POINT (-81.6538099999999929 30.3337100000000000)
432636,432636,34483,2022-10-27 06:44:18-04:00,2022-10-27 07:10:35-04:00,POINT (-81.6552099999999967 30.3289100000000005)
432637,432637,34483,2022-10-27 07:10:35-04:00,2022-10-27 07:42:26-04:00,POINT (-81.6506200000000035 30.3336199999999998)
432638,432638,34484,2022-10-28 00:47:04-04:00,2022-10-28 00:55:00-04:00,POINT (-81.6688213850000011 30.3685913700000008)


In [4]:
ID_list=sp['user_id'].unique()
len(ID_list)

11728

In [5]:
ndf = df[df['user_id'].isin(ID_list)]
ndf


,ID,user_id,latitude,longitude,flag,time
0,3865408,0,30.364965,-81.675575,True,2022-09-21 02:10:55
1,3865411,0,30.364965,-81.675575,True,2022-09-21 02:11:44
2,7902772,0,30.459910,-81.662560,True,2022-10-10 18:41:33
3,7902773,0,30.459920,-81.662550,True,2022-10-10 18:41:48
4,7902774,0,30.551570,-81.820950,True,2022-10-10 18:54:04
...,...,...,...,...,...,...
6715255,13978195,34490,30.297290,-81.562840,True,2022-10-24 11:19:43
6715256,13978196,34490,30.297290,-81.562840,True,2022-10-24 11:31:40
6715257,13978197,34490,30.297290,-81.562840,True,2022-10-24 11:49:18
6715258,13978198,34490,30.297290,-81.562840,True,2022-10-24 12:37:44


In [6]:
ndf.to_csv('newData.csv')

<h3>Add columns Lat-3856 and Lon-3857 with EPSG:3857</h3>

In [7]:
# Create a transformer object to convert from EPSG 4326 (WGS84) to EPSG 3857 (Web Mercator)
transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

# Iterate over rows and apply the transformer to convert the coordinates
for idx, row in tqdm(ndf.iterrows(), total=ndf.shape[0]):
    lat, lon = row['latitude'], row['longitude']
    x, y = transformer.transform(lon, lat)
    ndf.at[idx, 'Lat-3857'] = y
    ndf.at[idx, 'Lon-3857'] = x

# Print the updated DataFrame
print(ndf)

  0%|          | 0/6245748 [00:00<?, ?it/s]/var/folders/vy/m0tbb9cx2m176wk361h4ns900000gn/T/ipykernel_1700/1837106994.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf.at[idx, 'Lat-3857'] = y
/var/folders/vy/m0tbb9cx2m176wk361h4ns900000gn/T/ipykernel_1700/1837106994.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf.at[idx, 'Lon-3857'] = x
100%|██████████| 6245748/6245748 [11:15<00:00, 9244.64it/s]

               ID  user_id   latitude  longitude  flag                 time  \
0         3865408        0  30.364965 -81.675575  True  2022-09-21 02:10:55   
1         3865411        0  30.364965 -81.675575  True  2022-09-21 02:11:44   
2         7902772        0  30.459910 -81.662560  True  2022-10-10 18:41:33   
3         7902773        0  30.459920 -81.662550  True  2022-10-10 18:41:48   
4         7902774        0  30.551570 -81.820950  True  2022-10-10 18:54:04   
...           ...      ...        ...        ...   ...                  ...   
6715255  13978195    34490  30.297290 -81.562840  True  2022-10-24 11:19:43   
6715256  13978196    34490  30.297290 -81.562840  True  2022-10-24 11:31:40   
6715257  13978197    34490  30.297290 -81.562840  True  2022-10-24 11:49:18   
6715258  13978198    34490  30.297290 -81.562840  True  2022-10-24 12:37:44   
6715259  13978199    34490  30.297290 -81.562840  True  2022-10-24 13:49:11   

             Lat-3857      Lon-3857  
0        3.55

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
def home_location(input_df,s_hour=22,t_hour=6):
    '''
    ndf: gps data
    s_hour: define start time of night, defult = 22
    t_hour: define end time of night, defult = 6
    '''
    input_df['time']=pd.to_datetime(input_df['time'], infer_datetime_format=True)
    input_df['date']=input_df['time'].dt.date
    input_df['hour']=pd.Series(input_df['time']).dt.hour
    
    input_df['LAT_Grid']=np.round(input_df['Lat-3857']/20)*20  # Grid size = 20m
    input_df['LON_Grid']=np.round(input_df['Lon-3857']/20)*20
    id=input_df.iloc[0,0]
    input_df=input_df[(input_df['hour']>=s_hour)|(input_df['hour']<t_hour)]

    if len(input_df)==0:  # if no signal during the night, return NaN
        return np.nan,np.nan
    home = input_df.groupby(['LAT_Grid','LON_Grid'])['date'].nunique().idxmax()
    return home[0],home[1]

In [9]:
home=pd.DataFrame(columns=['ID','LAT','LON']) # initial a DataFrame to save home location

for i in tqdm(ID_list[0:]): 
    dffh=ndf[ndf['user_id']==i].sort_values('time',axis=0,ascending=True) # extract records
    h_lat,h_lon=home_location(dffh) # calculate home location
    home=home.append(pd.DataFrame([[i,h_lat,h_lon]],columns=['ID','LAT','LON'])) # append the new row 

100%|██████████| 11728/11728 [02:00<00:00, 97.45it/s] 


In [10]:
nan_df = home[home.isna().any(axis=1)]
home.dropna(subset=['LAT'], inplace=True)
home

,ID,LAT,LON
0,0,3550540.0,-9092080.0
0,2,3458860.0,-9093020.0
0,3,3456360.0,-9163720.0
0,8,3510020.0,-9105740.0
0,10,3565080.0,-9087560.0
...,...,...,...
0,34479,3473760.0,-9184540.0
0,34481,3530840.0,-9102880.0
0,34483,3503620.0,-9058160.0
0,34484,3551020.0,-9091340.0


<h3>----------------------------------------------------------------------------------------------------------------</h3>

<h3>
Further adjust to infer home location for more users</h3>

In [11]:
ID_list=nan_df['ID'].unique()
len(ID_list)

1773

In [12]:
#check if a date is in saturday or sunday.
def is_weekend(date):
    return date.weekday() in [5, 6]

def home_location(input_df):
    '''
    ndf: gps data
    '''
    input_df['time']=pd.to_datetime(input_df['time'], infer_datetime_format=True)
    input_df['date']=input_df['time'].dt.date
    input_df['hour']=pd.Series(input_df['time']).dt.hour
    
    input_df['LAT_Grid']=np.round(input_df['Lat-3857']/20)*20  # Grid size = 20m
    input_df['LON_Grid']=np.round(input_df['Lon-3857']/20)*20
    input_df= input_df[input_df['date'].apply(is_weekend)]
    
    if len(input_df)==0: 
        return np.nan,np.nan
    home2 = input_df.groupby(['LAT_Grid','LON_Grid']).count()['ID'].idxmax()
    return home2[0],home2[1]

In [13]:
home2=pd.DataFrame(columns=['ID','LAT','LON']) # initial a DataFrame to save home location

for i in tqdm(ID_list[0:]): 
    dffh=ndf[ndf['user_id']==i].sort_values('time',axis=0,ascending=True) # extract records
    h_lat,h_lon=home_location(dffh) # calculate home location
    home2=home2.append(pd.DataFrame([[i,h_lat,h_lon]],columns=['ID','LAT','LON'])) # append the new row 

100%|██████████| 1773/1773 [00:16<00:00, 106.88it/s]


In [14]:
home2.dropna(subset=['LAT'], inplace=True)
home2

,ID,LAT,LON
0,82,3373740.0,-9074520.0
0,83,3446280.0,-9044880.0
0,105,3513240.0,-9105460.0
0,142,3377220.0,-9178440.0
0,152,3559580.0,-9092660.0
...,...,...,...
0,34266,3473240.0,-9144240.0
0,34268,3464400.0,-9053100.0
0,34342,3526560.0,-9204640.0
0,34373,3541500.0,-9103240.0


In [15]:
new_home = pd.concat([home, home2], ignore_index=True)
new_home

,ID,LAT,LON
0,0,3550540.0,-9092080.0
1,2,3458860.0,-9093020.0
2,3,3456360.0,-9163720.0
3,8,3510020.0,-9105740.0
4,10,3565080.0,-9087560.0
...,...,...,...
10869,34266,3473240.0,-9144240.0
10870,34268,3464400.0,-9053100.0
10871,34342,3526560.0,-9204640.0
10872,34373,3541500.0,-9103240.0


In [16]:
# Create a transformer object to convert from EPSG 3857 (Web Mercator) to EPSG 4326 (WGS84)
inputGrid = pyproj.Proj(projparams='epsg:3857')
wgs84 = pyproj.Proj(projparams='epsg:4326')

new_home = new_home.reset_index(drop=True)

# Iterate over rows and apply the transformer to convert the coordinates
for idx, row in tqdm(new_home.iterrows(), total=new_home.shape[0]):
    lat, lon = pyproj.transform(inputGrid, wgs84, row['LON'], row['LAT'])
    new_home.at[idx, 'LAT-4326'] = lat
    new_home.at[idx, 'LON-4326'] = lon

# Print the updated DataFrame
new_home

100%|██████████| 10874/10874 [00:18<00:00, 603.17it/s]


,ID,LAT,LON,LAT-4326,LON-4326
0,0,3550540.0,-9092080.0,30.364891,-81.675544
1,2,3458860.0,-9093020.0,29.651722,-81.683988
2,3,3456360.0,-9163720.0,29.632203,-82.319097
3,8,3510020.0,-9105740.0,30.050323,-81.798254
4,10,3565080.0,-9087560.0,30.477524,-81.634940
...,...,...,...,...,...
10869,34266,3473240.0,-9144240.0,29.763921,-82.144106
10870,34268,3464400.0,-9053100.0,29.694962,-81.325381
10871,34342,3526560.0,-9204640.0,30.178849,-82.686688
10872,34373,3541500.0,-9103240.0,30.294798,-81.775796


In [17]:
new_home.to_csv('home_location.csv')